In [ ]:
# ==============================================================================

# 👩‍💻 Author    : Hyelim Jo
# 🎯 Purpose   : AI 윤리성 리스크 진단 에이전트 v1.0
# 📅 Created   : 2025-10-22
# 📜 Note      : evidence_collector.ipynb

# ==============================================================================


In [ ]:
# -------------------------------- Update Log ----------------------------------

# 2025-10-22 16:00 / 초기 생성 / Evidence Collector 기본 구조 구현
# 2025-10-22 16:30 / RAG 메모리 설계 / Baseline + Issue 메모리 분리
# 2025-10-22 17:00 / HuggingFace 임베딩 적용 / 경제성 개선

# ------------------------------------------------------------------------------


In [ ]:
# step1. 라이브러리 불러오기
import os
import json
import requests
from datetime import datetime, timedelta
from typing import Dict, List, Any
from bs4 import BeautifulSoup
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.documents import Document
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 불러오기 완료!")


In [1]:
# step2. 설정 및 경로 정의
# 데이터 경로 설정
base_dir = os.path.join("..", "ai_ethics_agent", "data")
reference_dir = os.path.join(base_dir, "reference")
crawled_dir = os.path.join(base_dir, "crawled")
processed_dir = os.path.join(base_dir, "processed")
baseline_embed_dir = os.path.join(base_dir, "embeddings", "baseline")
issue_embed_dir = os.path.join(base_dir, "embeddings", "issue")

# 디렉토리 생성
for dir_path in [crawled_dir, processed_dir, baseline_embed_dir, issue_embed_dir]:
    os.makedirs(dir_path, exist_ok=True)

print("✅ 경로 설정 완료!")


NameError: name 'os' is not defined

In [ ]:
# step3. 임베딩 모델 초기화 (HuggingFace - 경제성)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

print("✅ HuggingFace 임베딩 모델 초기화 완료!")


In [ ]:
# step4. Baseline 메모리 구축 (EU, OECD, UNESCO 문서)
def build_baseline_memory():
    """공식 문서 기반 Baseline 메모리 구축"""
    baseline_docs = []
    
    # PDF 파일 로드
    pdf_files = [
        "EU_AI_Act.pdf",
        "OECD_Privacy_2024.pdf", 
        "UNESCO_Ethics_2021.pdf"
    ]
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(reference_dir, pdf_file)
        if os.path.exists(pdf_path):
            loader = PyMuPDFLoader(pdf_path)
            docs = loader.load()
            # 메타데이터에 문서 타입 추가
            for doc in docs:
                doc.metadata["document_type"] = "baseline"
                doc.metadata["source"] = pdf_file
            baseline_docs.extend(docs)
            print(f"✅ {pdf_file} 로드 완료")
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    split_docs = text_splitter.split_documents(baseline_docs)
    
    # ChromaDB에 저장
    baseline_vectorstore = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding_model,
        persist_directory=baseline_embed_dir
    )
    
    print(f"✅ Baseline 메모리 구축 완료 ({len(split_docs)}개 청크)")
    return baseline_vectorstore

print("✅ Baseline 메모리 함수 정의 완료")


In [ ]:
# step5. 웹 크롤링 함수 정의
def crawl_web_content(keywords: List[str]) -> List[Dict[str, Any]]:
    """AI 윤리 관련 최신 뉴스/논문 크롤링"""
    crawled_data = []
    
    # 검색 키워드 조합
    search_queries = []
    for keyword in keywords:
        search_queries.extend([
            f"AI {keyword} 윤리 이슈",
            f"AI {keyword} 편향성 문제",
            f"AI {keyword} 개인정보보호",
            f"AI {keyword} 투명성"
        ])
    
    # 샘플 뉴스 데이터 (실제 구현 시 API 사용)
    sample_news = [
        {
            "title": "AI 채용 시스템의 편향성 문제 대두",
            "content": "최근 AI 기반 채용 시스템에서 성별, 연령, 인종에 따른 편향성 문제가 사회적 이슈로 대두되고 있습니다. 특히 이력서 분석 AI가 특정 집단에 불리하게 작동한다는 연구 결과가 발표되었습니다.",
            "source": "AI Ethics Today",
            "url": "https://example.com/ai-bias-hiring",
            "date": "2024-10-20",
            "category": "bias"
        },
        {
            "title": "개인정보보호법 개정, AI 서비스 규제 강화",
            "content": "개인정보보호법 개정안이 통과되어 AI 서비스의 데이터 수집 및 활용에 대한 규제가 강화되었습니다. 특히 개인정보 자동처리 시스템에 대한 사전 심의 절차가 도입되었습니다.",
            "source": "Privacy Weekly",
            "url": "https://example.com/privacy-law-update",
            "date": "2024-10-18",
            "category": "privacy"
        },
        {
            "title": "AI 알고리즘 투명성 확보 방안 논의",
            "content": "AI 알고리즘의 투명성 확보를 위한 정책 논의가 활발히 진행되고 있습니다. 특히 의사결정 과정의 설명 가능성을 높이는 방안들이 제시되고 있습니다.",
            "source": "Tech Policy News",
            "url": "https://example.com/ai-transparency",
            "date": "2024-10-15",
            "category": "transparency"
        }
    ]
    
    # 필터링: 1년 이내, 300자 이상, 신뢰할 수 있는 출처
    filtered_data = []
    cutoff_date = datetime.now() - timedelta(days=365)
    
    for news in sample_news:
        # 날짜 필터링 (1년 이내)
        news_date = datetime.strptime(news["date"], "%Y-%m-%d")
        if news_date >= cutoff_date:
            # 길이 필터링 (300자 이상)
            if len(news["content"]) >= 300:
                # 선정적 표현 필터링
                if not any(word in news["content"] for word in ["충격", "폭로", "clickbait"]):
                    filtered_data.append(news)
    
    print(f"✅ 웹 크롤링 완료 ({len(filtered_data)}개 문서)")
    return filtered_data

print("✅ 웹 크롤링 함수 정의 완료")


In [2]:
# step6. Issue 메모리 구축 (웹 크롤링 결과를 RAG에 저장)
def build_issue_memory(keywords: List[str]):
    """웹 크롤링 결과를 ChromaDB에 저장하여 장기 메모리 구축"""
    
    # 웹 크롤링 실행
    crawled_data = crawl_web_content(keywords)
    
    # Document 객체로 변환
    issue_docs = []
    
    for item in crawled_data:
        doc = Document(
            page_content=f"{item['title']}\n\n{item['content']}",
            metadata={
                "document_type": "issue",
                "source": item["source"],
                "url": item["url"],
                "date": item["date"],
                "category": item["category"],
                "title": item["title"]
            }
        )
        issue_docs.append(doc)
    
    # ChromaDB에 저장 (장기 메모리)
    if issue_docs:
        issue_vectorstore = Chroma.from_documents(
            documents=issue_docs,
            embedding=embedding_model,
            persist_directory=issue_embed_dir
        )
        print(f"✅ Issue 메모리 구축 완료 ({len(issue_docs)}개 문서)")
        return issue_vectorstore
    else:
        print("⚠️ 크롤링된 데이터가 없습니다.")
        return None

print("✅ Issue 메모리 구축 함수 정의 완료")


NameError: name 'List' is not defined

In [ ]:
# step7. 증거 수집 함수 정의 (가중치 8:2 적용)
def collect_evidence(service_profile: Dict[str, Any]) -> Dict[str, Any]:
    """서비스 프로파일 기반 증거 수집 (Baseline 0.8 : Issue 0.2)"""
    
    service_name = service_profile.get("service_name", "")
    risk_categories = service_profile.get("risk_categories", [])
    service_type = service_profile.get("service_type", "")
    
    print(f"\n🔍 증거 수집 시작: {service_name}")
    print(f"   - 서비스 유형: {service_type}")
    print(f"   - 리스크 카테고리: {risk_categories}")
    
    # Baseline 메모리 구축 (공식 문서)
    baseline_vectorstore = build_baseline_memory()
    
    # Issue 메모리 구축 (웹 크롤링 결과)
    issue_vectorstore = build_issue_memory(risk_categories)
    
    evidence_results = {
        "query": service_name,
        "weights": {"baseline": 0.8, "issue": 0.2},
        "scores": {},
        "baseline_sources": [],
        "issue_sources": []
    }
    
    # 각 리스크 카테고리별 증거 수집
    for category in risk_categories:
        query = f"{service_name} {category} 리스크 {service_type}"
        
        print(f"\n   📊 {category.upper()} 리스크 분석 중...")
        
        # Baseline 검색 (EU, OECD, UNESCO 문서)
        baseline_docs = baseline_vectorstore.similarity_search(query, k=3)
        baseline_score = 0.8  # 공식 문서 가중치
        
        # Issue 검색 (웹 크롤링 결과)
        issue_score = 0.0
        if issue_vectorstore:
            issue_docs = issue_vectorstore.similarity_search(query, k=2)
            issue_score = 0.2  # 웹 크롤링 가중치
        else:
            issue_docs = []
        
        # 종합 점수 계산 (가중치 적용)
        total_score = baseline_score + issue_score
        
        evidence_results["scores"][category] = total_score
        
        # Baseline 소스 추가 (공식 문서)
        for doc in baseline_docs:
            evidence_results["baseline_sources"].append({
                "content": doc.page_content[:300] + "...",
                "source": doc.metadata.get("source", "Unknown"),
                "category": category,
                "score": baseline_score,
                "document_type": "baseline"
            })
        
        # Issue 소스 추가 (웹 크롤링)
        for doc in issue_docs:
            evidence_results["issue_sources"].append({
                "content": doc.page_content[:300] + "...",
                "source": doc.metadata.get("source", "Unknown"),
                "category": category,
                "score": issue_score,
                "url": doc.metadata.get("url", ""),
                "date": doc.metadata.get("date", ""),
                "document_type": "issue"
            })
        
        print(f"      - Baseline 소스: {len(baseline_docs)}개")
        print(f"      - Issue 소스: {len(issue_docs)}개")
        print(f"      - 종합 점수: {total_score:.1f}")
    
    print(f"\n✅ 증거 수집 완료!")
    return evidence_results

print("✅ 증거 수집 함수 정의 완료")


In [ ]:
# step8. 테스트 실행
print("\n" + "="*60)
print("🔍 Evidence Collector 시작...")
print("="*60)

# Service Profiler에서 받은 결과 (예시)
test_service_profile = {
    "service_name": "이력서 분석 추천 시스템",
    "service_type": "recommendation", 
    "description": "채용 지원자의 이력서를 AI로 분석하여 적합한 후보자를 추천하는 시스템입니다.",
    "risk_categories": ["bias", "privacy", "transparency"]
}

print(f"\n📝 분석할 서비스: {test_service_profile['description']}...")

# 증거 수집 실행
evidence_result = collect_evidence(test_service_profile)

print("\n" + "="*60)
print("📊 증거 수집 결과 요약")
print("="*60)
print(f"   - 서비스명: {evidence_result['query']}")
print(f"   - 가중치: Baseline {evidence_result['weights']['baseline']} : Issue {evidence_result['weights']['issue']}")
print(f"   - 수집된 리스크: {list(evidence_result['scores'].keys())}")

print(f"\n📈 각 카테고리별 점수:")
for category, score in evidence_result['scores'].items():
    print(f"   - {category.upper()}: {score:.1f}")

print(f"\n📚 수집된 증거:")
print(f"   - Baseline 소스 (공식문서): {len(evidence_result['baseline_sources'])}개")
print(f"   - Issue 소스 (웹크롤링): {len(evidence_result['issue_sources'])}개")

print(f"\n🔗 다음 단계: Risk Assessor로 증거 전달 준비 완료")
print("="*60)
